In [2]:
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt
from Bio.SVDSuperimposer import SVDSuperimposer
import nglview as nv

# # Path: pymdna/__init__.py, prototype of the package and atomic is not properly referenced in the package at genertors.py now I just explicitly define the path loction
# import pymdna as mdna
import sys
sys.path.append('/Users/thor/surfdrive/Projects/pymdna/')
import pymdna as mdna 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


First we load the pdbs as is

In [3]:
loc = '../../pymdna/atomic/'
base_A, base_2AP = [md.load(loc+'BDNA_'+base+'.pdb') for base in ['A', '2AP']]
base_A, base_2AP

(<mdtraj.Trajectory with 1 frames, 21 atoms, 1 residues, without unitcells at 0x17b128050>,
 <mdtraj.Trajectory with 1 frames, 32 atoms, 1 residues, and unitcells at 0x17ac22090>)

Compare 2AP with A nuceleobase, find atoms that are a good fit (aka fit planar nucleobases on top of each other)
`['N1','N9', 'N3', 'C6', 'C4', 'C2', 'C5', 'N7']`

In [4]:
merge = base_A.stack(base_2AP)
view = nv.show_mdtraj(merge)
view

NGLWidget()

Now we use these atoms to move the non-canonical base to the same plane as the standard base A and compare the x,y,z coorindates of the atoms that are used for the ReferenceBase to construct the basis of the nucleobase. The fitted structure we can now add to the atomic directory ('BDNA_P.pdb'), and edit the `base_pair_map` `dict` in `generator.py` to encode the new base.

In [5]:
 
def get_rot_and_trans(subtraj_A,subtraj_B):
    """ fit only works now on a single frame (mdtraj returns xyz with shape (n_frames, atoms, xyz) 
        even for single frame trajs so hence the xyz[0]"""
    
    # load super imposer
    sup = SVDSuperimposer()

    # Set the coords, y will be rotated and translated on x
    x = subtraj_A.xyz[0]
    y = subtraj_B.xyz[0]
    sup.set(x, y)

    # Do the leastsquared fit
    sup.run()

    # Get the rms
    rms = sup.get_rms()

    # Get rotation (right multiplying!) and the translation
    rot, tran = sup.get_rotran()
    
    # now we have the instructions to rotate B on A
    return rot,tran,rms

def apply_superimposition(traj, rot, tran):
    
    # get xyz coordinates
    xyz = traj.xyz[0]
    
    # rotate subject on target
    new_xyz = np.dot(xyz, rot) + tran

    # replace coordinates of traj
    traj.xyz = new_xyz
    return traj

def fit_B_on_A(A, B, selection_A, selection_B):
    
    # create trajs containing only the selections
    subtraj_A = A.atom_slice(selection_A)
    subtraj_B = B.atom_slice(selection_B)

    # obtain instructions to rotate and translate B on A based on substraj structures
    rot, tran, rms = get_rot_and_trans(subtraj_A,subtraj_B)
    
    # do the superimposition of B on A and subsitute old with new xyz of B
    sup_B = apply_superimposition(B, rot, tran)

    # remove overlapping backbone atoms from B 
    selection_to_keep = [at.index for at in  B.top.atoms if at.index not in selection_B]

    new_B = sup_B
    return new_B, rms

def get_fit_indices(traj):
    residue = traj.top.residue(0)
    #print(residue, [at.name for at in residue._atoms])
    # res_indices = [at.index  for at in residue.atoms if at.name in ["P","C1'", "C5'",'N9','C4','N1']]
    res_indices = [at.index  for at in residue.atoms if at.name in ['N1','N9', 'N3', 'C6', 'C4', 'C2', 'C5', 'N7']]

    print(res_indices)
    return res_indices
            
def fit_bases(base_A, base_B):
    """ fit bases on trajectory """

    selection_A = get_fit_indices(base_A)
    selection_B = get_fit_indices(base_B)

    # Fits B on A
    return fit_B_on_A(A=base_A, B=base_B, selection_A=selection_A, selection_B=selection_B)


new_base_2AP, rms = fit_bases(base_A, base_2AP)
print(rms)
# Uncomment if you don't want hydrogens
#new_base_2AP.atom_slice(new_base_2AP.top.select('not element type hydrogen'))#.save('BDNA_P.pdb')
new_base_2AP.save_pdb('BDNA_P.pdb')


res_A = mdna.ReferenceBase(base_A)
res_ref = mdna.ReferenceBase(new_base_2AP)

print(res_A.base_type, res_ref.base_type)
print(res_A.C1_coords, res_ref.C1_coords)
print(res_A.N_coords, res_ref.N_coords)
print(res_A.C_coords, res_ref.C_coords)


[11, 13, 14, 15, 17, 18, 19, 20]
[12, 15, 16, 17, 22, 23, 24, 25]
0.004665368615066092
purine purine
[[-0.2479  0.5346 -0.    ]] [[-0.24228857  0.5235295  -0.00911196]]
[[-0.1291  0.4498 -0.    ]] [[-1.2690061e-01  4.4528759e-01 -3.2372773e-04]]
[[-0.1267  0.3124  0.    ]] [[-0.12616056  0.3066709  -0.00390143]]
